In [14]:
#Imports
import os
import xml.etree.ElementTree as ET
import pandas as pd
from datetime import datetime
import pickle
import tqdm
import line_profiler
from operator import itemgetter
from sklearn.preprocessing import OneHotEncoder
import math

In [2]:
def track_id_gen(dist, track):
    if track==None:
        return "NA"
    return track[0:4]+"-"+str(int(dist))

In [3]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")

In [4]:
race_ft_details = pd.DataFrame(pickle.load(open('race_details_ft.npy','rb')), columns=['@id', 'RaceNum', 'RaceName','RaceTime', 'Distance', 'RaceGrade', 'Track', 'date'])
race_ft_details['RaceId'] = race_ft_details['@id'] 
race_ft_details

,@id,RaceNum,RaceName,RaceTime,Distance,RaceGrade,Track,date,RaceId
0,501007024,1,FAST FREDDIE'S TYRE SERVICE,11:00AM,350m,Mixed 6/7,Healesville,01 Dec 19,501007024
1,498695480,2,SILVER EAGLE OUTFITTERS FINAL (VICGREYS),11:15AM,350m,Mixed 6/7 Final,Healesville,01 Dec 19,498695480
2,501007016,3,GRV VIC BRED SERIES HT1,11:32AM,350m,Grade 5 Heat,Healesville,01 Dec 19,501007016
3,501007017,4,GRV VIC BRED SERIES HT2,11:49AM,350m,Grade 5 Heat,Healesville,01 Dec 19,501007017
4,501007014,5,GRV VIC BRED SERIES HT3,12:07PM,350m,Grade 5 Heat,Healesville,01 Dec 19,501007014
...,...,...,...,...,...,...,...,...,...
150270,846624258,8,A1 SALVAGE & HARDWARE,08:55PM,405m,Mixed 4/5 Heat,Mandurah,01 Dec 22,846624258
150271,846624259,9,PRIME TROPHIES,09:17PM,405m,Mixed 4/5 Heat,Mandurah,01 Dec 22,846624259
150272,846624260,10,TOMMY SHELBY @ STUD,09:38PM,490m,Mixed 6/5,Mandurah,01 Dec 22,846624260
150273,846624261,11,GREYHOUNDFACTS.COM.AU,09:57PM,302m,Grade 5,Mandurah,01 Dec 22,846624261


In [5]:
dog_ft_details = pd.DataFrame(pickle.load(open('dog_details_ft.npy','rb')))
dog_ft_details.head(15)
dog_ft_details['DogId'] =   dog_ft_details['@id']

In [6]:
##Convert Margins:
dog_ft_details['SplitTimes']= dog_ft_details.SplitMargin.astype(float)
dog_ft_details['minMargin'] = dog_ft_details.groupby('RaceId')['SplitTimes'].transform('min')
dog_ft_details['SplitMargin'] = dog_ft_details.SplitTimes-dog_ft_details.minMargin
dog_ft_details['Margin1'] = dog_ft_details['Margin1'].fillna(0)
dog_ft_details['StartPrice'] = dog_ft_details['StartPrice'].fillna(0)

In [7]:
dog_ft_details.head(10)

,@id,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,...,Comments,SplitMargin,RunTime,Prizemoney,RaceId,TrainerId,TrainerName,DogId,SplitTimes,minMargin
0,314367490,1,SUPER BIG,6,6,32.0,$4.70,None,5.91,None,...,None,0.00,19.24,None,501007024,55811,J Boreland,314367490,6.58,6.58
1,296096022,2,RIO DIXIE,1,1,32.2,$2.80F,None,5.91,5.91,...,None,0.12,19.66,None,501007024,55811,J Boreland,296096022,6.70,6.58
2,304203706,3,COCO JUMBO,8,8,29.9,$3.70,None,6.99,1.07,...,None,0.17,19.73,None,501007024,22077,A Terry,304203706,6.75,6.58
3,291188715,4,BEARING GOLD,4,4,29.2,$32.00,None,8.46,1.47,...,None,0.23,19.84,None,501007024,103190,C Watson,291188715,6.81,6.58
4,317789815,5,ROLLIN' IN,3,3,28.2,$15.80,None,11.31,2.86,...,None,0.23,20.04,None,501007024,141797,M Craig,317789815,6.81,6.58
5,323686094,6,FAIRYTALE WISH,7,7,26.2,$8.40,None,12.44,1.13,...,None,0.12,20.11,None,501007024,67975,S Cachia,323686094,6.70,6.58
6,276160773,7,THANDIE BALE,5,10,29.7,$14.70,None,14.69,2.24,...,None,0.14,20.27,None,501007024,112793,S Collins,276160773,6.72,6.58
7,220432486,8,JAIMANDY ROY,2,2,30.7,$17.90,None,16.87,2.19,...,None,0.13,20.42,None,501007024,113625,P Donohue,220432486,6.71,6.58
8,340654458,S,RAIN DANCE,None,5,None,0,None,0,None,...,None,NaN,None,None,501007024,55811,J Boreland,340654458,NaN,6.58
9,251076165,R,MAX VOLTAGE,None,9,None,0,None,0,None,...,None,NaN,None,None,501007024,288227,M McClelland,251076165,NaN,6.58


In [9]:
full_details = pd.merge(race_ft_details,dog_ft_details, 'right', on='RaceId')

In [10]:
len(full_details)

1264032

In [ ]:
full_details.to_csv('full_race_details_ft.csv')

In [9]:

resultsdf = full_details
print(resultsdf.head())
split_distances = pd.read_csv("split_dist_ft.csv")
resultsdf["dist"] = resultsdf["Distance"].astype(str).str[:-1].astype(float)

resultsdf.run_time = resultsdf.RunTime.astype(float)
resultsdf["track_id"] = resultsdf.apply(lambda s: track_id_gen(s["dist"],s["Track"]), axis=1)
resultsdf_merged = pd.merge(resultsdf, split_distances, on=["track_id", "Track"])

       @id_x RaceNum                     RaceName RaceTime Distance  \
0  501007024       1  FAST FREDDIE'S TYRE SERVICE  11:00AM     350m   
1  501007024       1  FAST FREDDIE'S TYRE SERVICE  11:00AM     350m   
2  501007024       1  FAST FREDDIE'S TYRE SERVICE  11:00AM     350m   
3  501007024       1  FAST FREDDIE'S TYRE SERVICE  11:00AM     350m   
4  501007024       1  FAST FREDDIE'S TYRE SERVICE  11:00AM     350m   

   RaceGrade        Track       date     RaceId      @id_y  ... Checks  \
0  Mixed 6/7  Healesville  01 Dec 19  501007024  314367490  ...      0   
1  Mixed 6/7  Healesville  01 Dec 19  501007024  296096022  ...      0   
2  Mixed 6/7  Healesville  01 Dec 19  501007024  304203706  ...      1   
3  Mixed 6/7  Healesville  01 Dec 19  501007024  291188715  ...      2   
4  Mixed 6/7  Healesville  01 Dec 19  501007024  317789815  ...      8   

  Comments SplitMargin RunTime Prizemoney TrainerId TrainerName      DogId  \
0     None        0.00   19.24       None     5581

C:\Users\Nick\AppData\Local\Temp\ipykernel_24168\282841814.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  resultsdf.run_time = resultsdf.RunTime.astype(float)


In [7]:
betfair_df = pickle.load(open('df-betfairSP.npy', 'rb'))
betfair_df['dateF'] = pd.to_datetime(betfair_df.EVENT_DT, dayfirst=True).dt.date
betfair_df['dog_name'] = betfair_df.dog.str[1:].str.upper()
resultsdf['dateF']= pd.to_datetime(resultsdf.date).dt.date

NameError: name 'resultsdf' is not defined

In [18]:
# pickle Load results
# resultsdf = pd.DataFrame(pickle.load(open('race_details_ft.npy','rb')))
# resultsdf.head()
resultsdf = full_details
ohe = OneHotEncoder(dtype=int,sparse=False)
transformed = ohe.fit_transform(resultsdf[['Track']])
#enc = OneHotEncoder( categories=tracks)
print(resultsdf.shape)
print(len(transformed))
resultsdf["tracksOnehot"] = transformed.tolist()

split_distances = pd.read_csv("split_dist_ft.csv")
resultsdf["dist"] = resultsdf["Distance"].astype(str).str[:-1].astype(float)
resultsdf = resultsdf[resultsdf['RunTime'].notnull()]
resultsdf['run_time'] = pd.to_numeric(resultsdf['RunTime'])
resultsdf['split_margins'] = resultsdf.SplitMargin.astype(float)

resultsdf['place'] = resultsdf.Place.str[0].astype(float)
resultsdf["track_id"] = resultsdf.apply(
    lambda s: track_id_gen(s["dist"], s["Track"]), axis=1
)

resultsdf_merged = pd.merge(resultsdf, split_distances, on=["track_id", "Track"])
resultsdf_merged = resultsdf_merged[resultsdf_merged['RunTime'].notnull()]
resultsdf_merged["dateF1"] = pd.to_datetime(resultsdf_merged["date"], format="%d %b %y")
resultsdf_merged["StartPrice"] = pd.to_numeric(resultsdf_merged['StartPrice'].str[1:-1])
resultsdf_merged["speed"] = pd.to_numeric(resultsdf_merged["RunTime"]) / pd.to_numeric(
    resultsdf_merged["dist_x"]
)

resultsdf_merged["split_speed"] = pd.to_numeric(
    resultsdf_merged["SplitTimes"]
) / pd.to_numeric(resultsdf_merged["split_dist_estim"])

resultsdf_merged["box"] = pd.to_numeric(resultsdf_merged["Box"])
resultsdf_merged["margin"] = pd.to_numeric(resultsdf_merged["Margin1"])
resultsdf_merged["weigth"] = pd.to_numeric(resultsdf_merged["Weight"])
resultsdf_merged['split_margins'] = pd.to_numeric(resultsdf_merged.SplitMargin)
resultsdf_merged["dateF"] = pd.to_datetime(resultsdf_merged["date"], format="%d %b %y").dt.date
resultsdf_merged['dog_name'] = resultsdf_merged['DogName']

resultsdf_merged = pd.merge(
    resultsdf_merged, betfair_df, how="left", on=["dateF", "dog_name"]
)

form = resultsdf_merged.sort_values("dateF", ascending=False)
form.loc[form['place']==1, 'margin']=0
with open("results-df-merged-prices.npy", "wb") as fp:   #Pickling
    
    pickle.dump(form, fp)

    
print(form.head())

form = form.sort_values(['dateF'])
race_forms = form.groupby(["RaceId"])
dog_forms = form.groupby(["DogId"])
print(race_forms)
race_input_layer = []
race_classes = []
full_details_new = []
dog_stats_df = []
resultsdf_merged.head(50)
n = 0
for race, j in tqdm.tqdm(race_forms):
    try:
        race_date = pd.to_datetime(j["dateF"].iloc[0])
        dogs = j['DogId']
        race_id = race
        race_grade = j.RaceGrade.iloc[0]
        race_dist = j.dist_x.iloc[0]
        trackOHE = j.tracksOnehot.iloc[0]
        track_name = j.Track.iloc[0]
        dog_info = []
        num_features = 16
        

        #Starts off with blank outfits of what we need to fill, are then filled using box pos as indexer
        blank_stats = [-1]*num_features
        blank_race = [blank_stats]*8
        blank_classes = [8]*8
        blank_margins = [80]*8
        blank_sp = [0]*8
        blank_bsp = [0]*8
        blank_dogname = ["blank"]*8
        if len(dogs)>8:
            continue
        i = 0
        for dog in dogs:
            
            #print(f"dog id = {dog}")
            # if i ==1:
            #     print(f"{race_date=}")
            
            prev_form = dog_forms.get_group(dog)
            #print(f"{len(prev_form)=}")
            prev_form = prev_form[(prev_form["dateF"] < race_date)]
            #print(f"After date cut: {len(prev_form)=}")
            prev_races = len(prev_form)


            dog_speed_avg = prev_form.speed.mean()
            dog_speed_sd = prev_form.speed.std()
            dog_speed_max = prev_form.speed.max()
            dog_split_avg = prev_form.split_speed.mean()
            dog_split_sd = prev_form.split_speed.std()
            dog_split_max = prev_form.split_speed.max()
            dog_split_mg_avg = prev_form.split_margins.mean()
            dog_margin_mean = prev_form.margin.mean()
            dog_margin_sd = prev_form.margin.std()
            if prev_races>0:
                dog_margin_last = prev_form.margin.iloc[0]
                dog_speed_last = prev_form.speed.iloc[0]
                dog_race_count = prev_form.shape[0]
                try:
                    dog_wins = len(prev_form[prev_form["place"] == 1])
                    dog_places = len(prev_form[prev_form["place"] <= 3])
                except Exception as e:
                    print("no win")
            else:
                dog_margin_last = -1
                dog_speed_last = -1
                dog_race_count = 0
                dog_wins = 0
                dog_places = 0
            # print(dog_margin_last)
            #print()

            dog_box = j.box.iloc[i]
            dog_weight = j.weigth.iloc[i] # weight not in forms for prediction going to use last weight
            stats = [
                    dog_box,
                    dog_speed_avg,
                    dog_speed_sd,
                    dog_speed_max,
                    dog_split_avg,
                    dog_split_sd,
                    dog_split_max,
                    dog_split_mg_avg,
                    dog_margin_mean,
                    dog_margin_sd,
                    dog_race_count,
                    dog_wins,
                    dog_places,
                    dog_weight,
                    dog_speed_last,
                    dog_margin_last
                    ]
            #print(stats)
            stats = [-1 if math.isnan(x) else x for x in stats]
            dog_stats = (
                dog,
                stats,
                j.place.iloc[i],
                j.StartPrice.iloc[i],
                j.margin.iloc[i],
                j.BSP.iloc[i],
            )

            #Fill in the blank outputs
            idx = dog_box-1
            blank_race[idx] = stats
            blank_classes[idx] = j.place.iloc[i]
            blank_bsp[idx] = j.BSP.iloc[i]
            blank_margins[idx] = j.margin.iloc[i]
            blank_sp[idx] = j.StartPrice.iloc[i]
            blank_dogname[idx] = j.dog_name.iloc[i]




            dog_info.append(dog_stats)
            dog_stats_df.append([dog,
                    j.dog_name.iloc[i],
                    race_id,
                    race_grade,
                    race_date,
                    trackOHE,
                    track_name,
                    race_dist,
                    stats,
                    j.place.iloc[i],
                    j.StartPrice.iloc[i],
                    j.margin.iloc[i],
                    j.BSP.iloc[i],
                    j.RunTime.iloc[i]])
            #print(f"{dog_stats_df=}")
            i = i + 1
        
        

        dog_info.sort(key=itemgetter(0))
        # dog_stats_df.append(dog_info)
        input_layer = (
            trackOHE
            + [race_dist]
            + [item for sublist in blank_race for item in sublist]
        )
        
        # if len(dogs) > 8:
        #     print(f"{race_id=}")
        #     print(f"{race_date=}")
        #     print(f"{j.track_x.iloc[1]=}")
        classes = blank_classes
        prices = blank_sp
        margins = blank_margins
        betfairSP = blank_bsp
        dogname = blank_dogname
        race_input_layer.append(input_layer)
        race_classes.append(classes)
        full_details_new.append(
            (race_id, input_layer, classes, prices,  margins, betfairSP, dogname)
        )

        
    except Exception as e:
        print(f"{e=}")
    # n += 1
    # if n >10:
    #     break

(1264032, 32)
1264032


C:\Users\Nick\AppData\Local\Temp\ipykernel_13364\4022833189.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsdf['run_time'] = pd.to_numeric(resultsdf['RunTime'])
C:\Users\Nick\AppData\Local\Temp\ipykernel_13364\4022833189.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsdf['split_margins'] = resultsdf.SplitMargin.astype(float)
C:\Users\Nick\AppData\Local\Temp\ipykernel_13364\4022833189.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

             @id_x RaceNum                  RaceName RaceTime Distance  \
171235   849837292       8       SOUTHERN SHEETMETAL  08:16PM     545m   
1000615  846620229       8  GREYHOUNDS AS PETS STAKE  09:04PM     530m   
1000604  846620228       7    @THEDOGSSA MIXED STAKE  08:40PM     530m   
1000605  846620228       7    @THEDOGSSA MIXED STAKE  08:40PM     530m   
1000606  846620228       7    @THEDOGSSA MIXED STAKE  08:40PM     530m   

         RaceGrade       Track       date     RaceId      @id_y  ... PPMAX  \
171235   Mixed 4/5    Ballarat  01 Dec 22  849837292  558491864  ...   NaN   
1000615    Grade 5  Angle Park  01 Dec 22  846620229  592709559  ...   NaN   
1000604  Mixed 4/5  Angle Park  01 Dec 22  846620228  529306503  ...   NaN   
1000605  Mixed 4/5  Angle Park  01 Dec 22  846620228  520620677  ...   NaN   
1000606  Mixed 4/5  Angle Park  01 Dec 22  846620228  570629991  ...   NaN   

        PPMIN IPMAX IPMIN MORNINGTRADEDVOL  PPTRADEDVOL IPTRADEDVOL  \
171235    NaN  

100%|██████████| 143231/143231 [45:56<00:00, 51.95it/s] 


full_details

In [ ]:
testdf = pd.DataFrame(race_input_layer)
print(testdf.shape)
no_nans = testdf.dropna()
print(no_nans.shape)

In [ ]:
testdf

In [ ]:
len(dog_stats_df)

In [ ]:
with open("142k fasttrack FFNN new margins.npy", "wb") as fp:   #Pickling
    
    pickle.dump(full_details_new, fp)

In [ ]:
file = open(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\DATA\total_list_new_box_index.npy", "rb")
data = pickle.load(file)
# seperate out classes from inputs
raceIDs, inputs, classes, prices, margins, betfairSP,names = zip(*data)
for i, race_id in enumerate(raceIDs):
    #print(i)
    if race_id =="140918784":
        first_arg = classes
        ins = inputs[i]
        cl = classes[i]
        dognames = names[i]
        print(i,race_id,cl, dognames)
        x_test = resultsdf[resultsdf['race_id']==race_id]
        # print(i,race_id)
        # print(prices[i])
        break
        

In [ ]:
print(full_details_new)

In [ ]:
raceIDs, inputs, classes, prices, margins, betfairSP = zip(*pickle.load(open('total_list_new_box_test.npy', 'rb')))

In [ ]:
for i, race_id in enumerate(raceIDs):
    if race_id =="140918784":
        first_arg = classes
        ins = inputs[i]
        cl = classes[i]
        print(i,race_id)
        x_test = resultsdf[resultsdf['race_id']==race_id]
        # print(i,race_id)
        # print(prices[i])
        break
        

In [ ]:
raceIDs[1285]

In [ ]:
cl

In [ ]:
resultsdf[resultsdf['race_id']=='128662384']

In [19]:
dfx = pd.DataFrame(dog_stats_df, columns=['dogid','dog_name','raceid','race_grade','date','trackOHE','track_name','dist','stats','place','startprice','margin','bfSP', 'runtime'])
dfx.head(10)

,dogid,dog_name,raceid,race_grade,date,trackOHE,track_name,dist,stats,place,startprice,margin,bfSP,runtime
0,214382420,SUPERIOR POWER,498695480,Mixed 6/7 Final,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Healesville,350.0,"[4, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",8.0,1.9,6.87,2.280000,19.87
1,302937173,SUPERIOR DUNKLEY,498695480,Mixed 6/7 Final,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Healesville,350.0,"[3, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",7.0,37.3,5.74,28.858711,19.79
2,296096015,WORTHY PRINCE,498695480,Mixed 6/7 Final,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Healesville,350.0,"[6, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",6.0,19.1,3.66,17.287920,19.64
3,296096016,ANNIHILATION,498695480,Mixed 6/7 Final,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Healesville,350.0,"[1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",4.0,7.9,2.01,10.456086,19.53
4,291188717,THURSDAY THUNDER,498695480,Mixed 6/7 Final,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Healesville,350.0,"[8, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",3.0,38.6,1.93,40.000000,19.52
5,314367487,RIO HUSTLER,498695480,Mixed 6/7 Final,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Healesville,350.0,"[2, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",2.0,6.3,0.89,7.200000,19.45
6,314368991,CALLUM ARI,498695480,Mixed 6/7 Final,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Healesville,350.0,"[5, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",1.0,11.5,0.00,19.500000,19.39
7,282125654,GRUMPY DEREK,498695480,Mixed 6/7 Final,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Healesville,350.0,"[7, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",5.0,5.4,3.33,7.543589,19.62
8,328356413,CRAZY OLD BIRD,500539128,Maiden Final,2019-12-03,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Horsham,410.0,"[6, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",8.0,14.6,19.74,11.000000,24.67
9,331308298,SILVER COMPASS,500539128,Maiden Final,2019-12-03,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Horsham,410.0,"[2, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",7.0,40.6,15.69,110.000000,24.38


In [15]:
len(dfx['stats'][5])

16

In [13]:
with open("dog_stats_df_FASTTRACK new margins.npy", "wb") as fp:   #Pickling
    
    pickle.dump(dfx, fp)

In [ ]:
file = open( "total_list.npy", 'rb')
data = pickle.load(file)
raceIDs, inputs, classes, prices, win_price, margins, betfairSP = zip(*data)
betfairSP

In [ ]:
bfSPdf = pd.DataFrame(betfairSP)
bfSPdf